In [10]:
from tensorflow.keras.datasets import boston_housing
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Veriyi Yüklemek

In [11]:
(train_data,train_targets),(test_data,test_targets) = boston_housing.load_data()

In [4]:
train_data.shape

(404, 13)

In [12]:
test_data.shape

(102, 13)

In [13]:
#  Normalize = [ x(i) - avg(mean) ] / std(x)

mean = train_data.mean(axis = 0)
std = train_data.std(axis = 0)

train_data -= mean
train_data /= std

test_data -= mean
test_data /= std

In [14]:
from tensorflow.keras import models
from tensorflow.keras import layers

def BuildModel():
    model = models.Sequential()
    model.add(layers.Dense(64,activation="relu",input_shape = (train_data.shape[1],)))
    model.add(layers.Dense(64,activation="relu"))
    model.add(layers.Dense(1))

    model.compile(
        optimizer = "rmsprop",
        loss = "mse",
        metrics = ["mae"]
    )

    return model
    

In [ ]:
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

for i in range(k):
    print("processing fold",i+1)

    # Validation Data
    val_data = train_data[i * num_val_samples : (i+1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples : (i+1) * num_val_samples]

    # Train Dataset
    partial_train_data = np.concatenate([train_data[:i * num_val_samples],train_data[(i + 1) * num_val_samples :]],axis=0)
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],train_targets[(i + 1) * num_val_samples :]],axis=0)


    # Keras Modelini Derler
    model = BuildModel()
    model.fit(partial_train_data,partial_train_targets,epochs = num_epochs,batch_size=1,verbose = 1)

    # Doğrulama ver setinde değerlendirir.
    val_mse,val_mae = model.evaluate(val_data,val_targets)
    all_scores.append(val_mae)

processing fold 1
Epoch 1/100
303/303 [==============================] - 2s 3ms/step - loss: 233.4196 - mae: 11.7942
Epoch 2/100
303/303 [==============================] - 1s 3ms/step - loss: 30.8815 - mae: 3.7539
Epoch 3/100
303/303 [==============================] - 1s 3ms/step - loss: 22.6540 - mae: 3.1544
Epoch 4/100
303/303 [==============================] - 1s 3ms/step - loss: 19.0561 - mae: 2.8558
Epoch 5/100
303/303 [==============================] - 1s 3ms/step - loss: 16.9172 - mae: 2.7562
Epoch 6/100
303/303 [==============================] - 1s 3ms/step - loss: 16.0714 - mae: 2.6391
Epoch 7/100
303/303 [==============================] - 1s 3ms/step - loss: 15.0730 - mae: 2.5735
Epoch 8/100
303/303 [==============================] - 1s 3ms/step - loss: 14.1441 - mae: 2.5353
Epoch 9/100
303/303 [==============================] - 1s 3ms/step - loss: 13.3723 - mae: 2.4197
Epoch 10/100
303/303 [==============================] - 1s 3ms/step - loss: 12.9975 - mae: 2.3760
Epoch 11/

In [ ]:
all_scores

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor="val_mae",
    mode="min",
    verbose=1,
    patience=20
)

In [ ]:
num_epochs = 500
all_mae_histories = []

for i in range(k):
    print("processing fold ",i+1)

    # Validation Data
    val_data = train_data[i * num_val_samples : (i+1) * num_val_samples]
    val_targets = train_targets[i * num_val_samples : (i+1) * num_val_samples]

    # Train Dataset
    partial_train_data = np.concatenate([train_data[:i * num_val_samples],train_data[(i + 1) * num_val_samples :]],axis=0)
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],train_targets[(i + 1) * num_val_samples :]],axis=0)

    model = BuildModel()

    history = model.fit(
        partial_train_data,
        partial_train_targets,
        validation_data=(val_data,val_targets),
        epochs=num_epochs,
        callbacks=[early_stopping],
        batch_size = 1,
        verbose = 1)

    mae = history.history["mae"]
    val_mae = history.history["val_mae"]

    all_mae_histories.append(val_mae)
    

In [ ]:
loss = history.history["loss"]
val_loss = history.history["val_loss"]

epochs = range(1,len(loss) + 1)

plt.plot(epochs,loss,"b",label = "Eğitim Kaybı")
plt.plot(epochs,val_loss,"ro",label = "Validation Kaybı")
plt.xlabel("İterasyon")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

plt.figure()

plt.plot(epochs,mae,"b",label = "Eğitim için MAE")
plt.plot(epochs,val_mae,"ro",label = "Validation için MAE")
plt.xlabel("İterasyon")
plt.ylabel("MAE")
plt.legend()
plt.grid(True)
plt.show()